# **LAB 5**
______

## **Zad 1**
> #### `np` - numer procesu
> #### `lp` - liczba procesów
> #### `n` - indeks w szeregu (od 1)

In [138]:
%%sh
cat > pi-mpi.c << EOF


#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

int main(int argc, char *argv[]) {
	int i,np,lp;
	int tag=50;
	MPI_Status status;
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &np);
	MPI_Comm_size(MPI_COMM_WORLD, &lp);

	if (np == 0) {
		double suma = 0, s=0, n = 1.0;
    printf("\nObliczanie równoległe liczby PI z wzoru Leibniza dla %d procesów", lp);
		for(i=0;i<lp;i++)
		{
      MPI_Send(&n, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
      n++;
		}

		for(i=0;i<lp;i++)
		{
			MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
			suma+=s;
		}
		printf("\nWynik = %lf\n", 4 * suma );
	}
	else
	{
		double s;
    double n;

    MPI_Recv(&n, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);

		s = pow(-1.0, n-1) / (2*n-1);
		MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD);
	}
	MPI_Finalize();
}
EOF
mpicc pi-mpi.c -lm && mpirun -n 128 --allow-run-as-root a.out 


Obliczanie równoległe liczby PI z wzoru Leibniza dla 128 procesów
Wynik = 3.133780


## **Zad 2**

> #### `np` - numer procesu
> #### `lp` - liczba procesów
> #### `dx` - odległość między sąsiednimi punktami ( wysokość trapezu )
> #### `xp` - początek przedziału
> #### `xk` - koniec przedziału

Metoda *funkcja* - funkcja podcałkowa

In [41]:

%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

double funkcja(double x) {
	return x;
}

int main(int argc, char *argv[]) {
	int i,np,lp;
	int tag=50;
	MPI_Status status;
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &np);
	MPI_Comm_size(MPI_COMM_WORLD, &lp);

	if (np == 0) {
		double xp,xk,pole,s=0,dx;
		printf("\nObliczanie calki oznaczonej metoda prostokątów dla funkcji f(x)=x");
		xp = 1.0;
		xk = 4.0;
		dx=(xk-xp)/(double)lp;

		// wysylanie wartosci ci xp i xk oraz dx do poszczegolnych procesow
		for(i=0;i<lp;i++)
		{
			MPI_Send(&xp, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
			MPI_Send(&xk, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
			MPI_Send(&dx, 1, MPI_DOUBLE, i,tag, MPI_COMM_WORLD);
		}

	  // odbieranie oraz sumowanie wyników z wszystkich procesów
		for(i=0;i<lp;i++)
		{
			MPI_Recv(&s, 1, MPI_DOUBLE, i, tag,MPI_COMM_WORLD, &status);
			pole+=s;
		}
		printf("\nPole pod calka wynosi = %lf\n", dx * (pole + (funkcja(xp) + funkcja(xk))/2) );
	}
	else
	{
		double s,dx,xp,xk;

		// odbieranie wartości od procesu 0
		MPI_Recv(&xp, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
		MPI_Recv(&xk, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);
		MPI_Recv(&dx, 1, MPI_DOUBLE, 0, tag,MPI_COMM_WORLD, &status);

		// obliczanie pola trapezu dla podanej funkcji
		s = funkcja(xp + np * dx); 

		//wysyłanie pola trapezu do procesu 0
		MPI_Send(&s, 1, MPI_DOUBLE, 0,tag, MPI_COMM_WORLD);
	}
	MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 8 --allow-run-as-root a.out


Obliczanie calki oznaczonej metoda prostokątów dla funkcji f(x)=x
Pole pod calka wynosi = 7.875000


## **Zad 3**
Proces 0 to proces Root, który rozdziela wiersze
i kolumny macierzy B, C pomiędzy procesy robocze Slave wykonujące 
obliczenia cząstkowe mnożenia macierzy. 

Procesy Slave zwracają wyniki 
do procesu Root, z których składa macierz wynikową A.



In [69]:
%%sh
cat > pi-mpi.c << EOF

#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>
#define N 4

MPI_Status status;
double A[N][N],B[N],C[N][N];

int main(int argc, char **argv) {
	int processCount, processId, slaveTaskCount, source, dest, rows, offset;
	struct timeval start, stop;
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &processId);
	MPI_Comm_size(MPI_COMM_WORLD, &processCount);

	slaveTaskCount = processCount - 1;
	if (processId == 0) {
		double start = MPI_Wtime();
		srand ( time(NULL) );

		for (int i = 0; i<N; i++) {
			for (int j = 0; j<N; j++) {
				A[i][j]= rand()%10;
				B[j]= rand()%10;
			}
		}

		printf("\nMnozenie macierzy za pomoca MPI \n");
		printf("\nMacierz A\t\t\tWektor B\n\n");

		for (int i = 0; i<N; i++) {
			for (int j = 0; j<N; j++) {
				printf("%.0f\t", A[i][j]);
			}
      printf("| %.0f\t\n", B[i]);
		}

    for (int i = 0; i<N; i++) {
      printf("_\t");
		}
    printf("\n");

    for (int i = 0; i<N; i++) {
      printf("%.0f\t", B[i]);
		}

    printf("\n");
		rows = N/slaveTaskCount;
		offset = 0;

		for (dest=1; dest <= slaveTaskCount; dest++)
		{
			MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);

			MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);

			MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);

			MPI_Send(&B, N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
			offset = offset + rows;
		}
		for (int i = 1; i <= slaveTaskCount; i++)
		{
			source = i;

			MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);

			MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
      
			MPI_Recv(&C[offset][0], rows*N, MPI_DOUBLE, source, 2 , MPI_COMM_WORLD, &status);
		}

		printf("\nWynikowa macierz C = A * B:\n\n");
		for (int i = 0; i<N; i++) {
			for (int j = 0; j<N; j++)
				printf("%.0f\t", C[i][j]);
			printf ("\n");
		}
		printf ("\n");
		double end = MPI_Wtime();
		printf("Czas obliczen %f",end - start);
	} 

	if (processId > 0) {
		source = 0;
		MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);

		MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);

		MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);

		MPI_Recv(&B, N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);

		for (int i = 0; i<rows; i++) {
      for (int j = 0; j<N; j++)
				C[i][j] = A[i][j] * B[j];
		}

		MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

		MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

		MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
	}
	MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 3 --allow-run-as-root a.out


Mnozenie macierzy za pomoca MPI 

Macierz A			Wektor B

9	8	4	9	| 5	
1	4	0	8	| 2	
1	6	4	5	| 4	
2	0	2	3	| 1	
_	_	_	_	
5	2	4	1	

Wynikowa macierz C = A * B:

45	16	16	9	
5	8	0	8	
5	12	16	5	
10	0	8	3	

Czas obliczen 0.000391

# **Lab 6**
____

In [96]:

%%sh
cat > pi-mpi.c << EOF

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <string.h>
#include "mpi.h"


#define DEFAULT_ITERATIONS 64
#define GRID_WIDTH 256
#define DIM 16 // assume a square grid

int mod(int a, int b)
{
    int r = a % b;
    return r < 0 ? r + b : r;
}

int main(int argc, char **argv)
{

    int global_grid[256] =
        {0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0};

    // MPI Standard variable
    int num_procs;
    int ID, j;
    int iters = 0;
    int num_iterations;

    // Setup number of iterations
    if (argc == 1)
    {
        num_iterations = DEFAULT_ITERATIONS;
    }
    else if (argc == 2)
    {
        num_iterations = atoi(argv[1]);
    }
    else
    {
        printf("Usage: ./gameoflife <num_iterations>\n");
        exit(1);
    }

    // Messaging variables
    MPI_Status stat;

    // MPI Setup
    if (MPI_Init(&argc, &argv) != MPI_SUCCESS)
    {
        printf("MPI_Init error\n");
    }

    MPI_Comm_size(MPI_COMM_WORLD, &num_procs); // Set the num_procs
    MPI_Comm_rank(MPI_COMM_WORLD, &ID);

    assert(DIM % num_procs == 0);

    // Setup environment
    int *arr = (int *)malloc(DIM * ((DIM / num_procs) + 2) * sizeof(int));
    for (iters = 0; iters < num_iterations; iters++)
    {
        //printf("%d %d\n",ID, DIM * ((DIM / num_procs) + 2));
        j = DIM;
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            arr[j] = global_grid[i];
            // if(ID==1)
            //     printf(" %d %d \n",j,i);
            j++;
        }

        if (num_procs != 1)
        {
            //odd-even send_recv
            int incoming_1[DIM];
            int incoming_2[DIM];
            int send_1[DIM];
            int send_2[DIM];
            if (ID % 2 == 0)
            {

                //first16
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                    // printf(" - %d - ",send_1[i]);
                    //printf(" %d %d\n ",i,i+DIM);
                }
                //first row to ID-1
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);

                //last16
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                    // printf(" %d %d\n ",i,(DIM * (DIM / num_procs)) + i);
                }
                //last row to ID+1
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            else
            {
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            if (ID % 2 == 0)
            {
                MPI_Recv(&incoming_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD, &stat);
                MPI_Recv(&incoming_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD, &stat);
            }
            else
            {
                //first16
                for (int i = 0; i < DIM; i++)
                {
                    send_1[i] = arr[i + DIM];
                }
                MPI_Ssend(&send_1, DIM, MPI_INT, mod(ID - 1, num_procs), 1, MPI_COMM_WORLD);

                //last16
                for (int i = 0; i < DIM; i++)
                {
                    send_2[i] = arr[(DIM * (DIM / num_procs)) + i];
                }
                MPI_Ssend(&send_2, DIM, MPI_INT, mod(ID + 1, num_procs), 1, MPI_COMM_WORLD);
            }
            for (int i = 0; i < DIM; i++)
            {
                arr[i] = incoming_1[i];
                arr[(DIM * ((DIM / num_procs) + 1)) + i] = incoming_2[i];
            }
        }
        else
        {
            for (int i = 0; i < DIM; i++)
            {
                arr[i + GRID_WIDTH + DIM] = global_grid[i];
                //printf(" %d %d \n",i + GRID_WIDTH+DIM,i);
            }
            for (int i = GRID_WIDTH; i < GRID_WIDTH + DIM; i++)
            {
                arr[i - GRID_WIDTH] = global_grid[i - DIM];
                //printf(" %d %d \n",i - GRID_WIDTH,i-DIM);
            }
        }
        //game logic neighbours
        int * final = (int *)malloc(DIM * ((DIM / num_procs)) * sizeof(int));

        for (int k = DIM; k < DIM * ((DIM / num_procs) + 1); k++)
        {
            int total_rows = DIM * (DIM / num_procs) + 2;
            int r = k / DIM;
            int c = k % DIM;
            int prev_r = mod(r - 1, total_rows);
            int prev_c = mod(c - 1, DIM);
            int next_r = mod(r + 1, total_rows);
            int next_c = mod(c + 1, DIM);

            int count = arr[prev_r * DIM + prev_c] + arr[prev_r * DIM + c] + arr[prev_r * DIM + next_c] + arr[r * DIM + prev_c] + arr[r * DIM + next_c] + arr[next_r * DIM + prev_c] + arr[next_r * DIM + c] + arr[next_r * DIM + next_c];
            if (arr[k] == 1)
            {
                if (count < 2)
                    final[k - DIM] = 0;
                else if (count > 3)
                    final[k - DIM] = 0;
                else
                    final[k - DIM] = 1;
            }
            else
            {
                if (count == 3)
                    final[k - DIM] = 1;
                else
                    final[k - DIM] = 0;
            }
        }

        j = 0;
        for (int i = ID * (GRID_WIDTH / num_procs); i < (ID + 1) * (GRID_WIDTH / num_procs); i++)
        {
            global_grid[i] = final[j];
            j++;
        }
        MPI_Gather(final, DIM * (DIM / num_procs), MPI_INT, &global_grid, DIM * (DIM / num_procs), MPI_INT, 0, MPI_COMM_WORLD);

        // Output the updated grid state
        if (ID == 0)
        {
            printf("\nIteracja nr %d:\n", iters);
            for (j = 0; j < GRID_WIDTH; j++)
            {
                if (j % DIM == 0)
                {
                    printf("\n");
                }
                if(global_grid[j] == 1)
                  printf("*  ");
                else
                  printf("  ");
            }
            printf("\n");
        }
    }

    // Clean up memory
    free(arr);
    MPI_Finalize(); // finalize so I can exit
}

//mpirun -n <num_procs> gameoflife <num_iters>
//mpirun -n 8 gameoflife 16
EOF
mpicc pi-mpi.c && mpirun -n 2 --allow-run-as-root a.out


Iteracja nr 0:

                                
*    *                            
  *  *                            
  *                              
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                

Iteracja nr 1:

                                
    *                            
*    *                            
  *  *                            
                                
                                
                                
                                
                                
                                
                                
                                
                                